In [1]:
from neo4j import GraphDatabase, Result
import random
import pyvis
import networkx as nx
from IPython.display import display, HTML
from pathlib import Path
from string import ascii_uppercase, ascii_lowercase
import datetime
import time
import json
from pathlib import Path
from tqdm import tqdm
import pandas as pd

# 0. Генерация синтетических данных

#### Описание данных
Есть 3 типа сущностей:
1. Исследователи. У исследоваетелй есть уникальное имя, организация и возраст
2. Носители. У носителей есть уникальный четырёхбуквенный код, возраст, гендер и родной город
3. Записи. У записей есть уникальный id, длина, дата и место

Есть 5 типов связи:
1. Записал (исследователь -> запись)
2. Был записан (носитель -> запись)
3. Родственник (носитель -> носитель), 2 типа: родитель и ребёнок
4. Руководит (исследователь -> исследователь)
5. Руководим (исследователь -> исследователь)

```python
getname = lambda: ''.join(random.choices(ascii_lowercase, k=random.randint(4,10))).capitalize()
getcode = lambda: ''.join(random.choices(ascii_uppercase, k=4))
getage = lambda: random.randint(18, 59)
gender = lambda: random.choices(['m', 'f', 'n'], weights=[0.45, 0.45, 0.1], k=1)[0]
getcity = lambda: random.choice(ascii_uppercase)
getid = lambda: random.randint(0, 99999)
getlen = lambda: random.randint(30, 5 * 3600)
getorg = lambda: ''.join(random.choices(ascii_uppercase, k=2))
getdate = lambda: datetime.date.fromordinal(random.randint(736299, 738946)).strftime('%d.%m.%Y')
```

```python
researchers = {}
recordings = {}
speakers = {}
recorded = set()
was_recorded = set()
related = set()
tutors = set()
```

```python
while len(researchers) < 10:
    name = getname()
    researchers[name] = {'name': name, 'age': getage(), 'organisation': getorg()}

while len(recordings) < 50:
    id_ = getid()
    recordings[id_] = {'id': id_, 'location': getcity(), 'date': getdate(), 'length': getlen()}

while len(speakers) < 20:
    code = getcode()
    speakers[code] = {'code': code, 'age': getage(), 'born in': getcity(), 'gender': gender()}
```

```python
researchers_list = sorted(researchers)
recordings_list = sorted(recordings)
speakers_list = sorted(speakers)
```

```python
while len(recorded) < 50:
    recorded.update([(random.choice(researchers_list), random.choice(recordings_list))])

while len(was_recorded) < 70:
    was_recorded.update([(random.choice(speakers_list), random.choice(recordings_list))])

while len(related) < 10:
    related.update([tuple(random.sample(speakers_list, 2))])

while len(tutors) < 4:
    tutors.update([tuple(random.sample(researchers_list, 2))])
```

```python
json_data = {
    'researchers': list(researchers.values()),
    'recordings': list(recordings.values()),
    'speakers': list(speakers.values()),
    'recorded': list(recorded),
    'was_recorded': list(was_recorded),
    'related': list(related),
    'tutors': list(tutors)
}
```

```python
json.dump(json_data, Path('data.json').open(encoding='utf-8', mode='w'), indent='\t')
```

# 1. Подключение к БД

In [2]:
URI = "neo4j://localhost:7687"
AUTH = ("neo4j", "password")

In [3]:
driver = GraphDatabase.driver(URI, auth=AUTH)

In [4]:
driver.execute_query('''MATCH (n) DETACH DELETE n''')

EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x0000023E3085F650>, keys=[])

# 2. Запись данных

## 2.1. Функции

In [5]:
write_name = lambda name: name if len(name.split()) == 1 else f'`{name}`'

In [6]:
def write_params(parameters):
    if len(parameters) == 0:
        return ''
    key_arguments = []
    for key, value in parameters.items():
        out_value = value if not isinstance(value, str) else f'"{value}"'
        key_arguments.append(f'{write_name(key)}: {out_value}')
    return ' {' + ', '.join(key_arguments) + '}'

In [7]:
def write_node(nodetype, parameters, driver=driver):
    driver.execute_query(f'CREATE (n:{write_name(nodetype)}{write_params(parameters)})')

In [8]:
def write_link(nodetype_from, nodetype_to, node_identify_from, node_identify_to, link_type, link_parameters={}, driver=driver):
    driver.execute_query(f'''
    MATCH (n1:{write_name(nodetype_from)}{write_params(node_identify_from)}),
    (n2:{write_name(nodetype_to)}{write_params(node_identify_to)})
    CREATE (n1)-[:{write_name(link_type)}{write_params(link_parameters)}]->(n2)
    ''')

## 2.2. Конфигурация и запись

In [9]:
data = json.load(Path('data.json').open(encoding='utf-8'))

In [10]:
for res in tqdm(data['researchers']):
    write_node('Researcher', res)

for rec in tqdm(data['recordings']):
    rec['date'] = rec['date'][-4:] + '-' + rec['date'][-7:-5] + '-' + rec['date'][:2]
    write_node('Recording', rec)

for speaker in tqdm(data['speakers']):
    write_node('Speaker', speaker)

for researcher, recording in tqdm(data['recorded']):
    write_link('Researcher', 'Recording', {'name': researcher}, {'id': recording}, 'recorded')

for speaker, recording in tqdm(data['was_recorded']):
    write_link('Speaker', 'Recording', {'code': speaker}, {'id': recording}, 'was_recorded')

for speaker_1, speaker_2 in tqdm(data['related']):
    write_link('Speaker', 'Speaker', {'code': speaker_1}, {'code': speaker_2}, 'related', {'type': 'parent'})
    write_link('Speaker', 'Speaker', {'code': speaker_2}, {'code': speaker_1}, 'related', {'type': 'child'})

for researcher_1, researcher_2 in tqdm(data['tutors']):
    write_link('Researcher', 'Researcher', {'name': researcher_1}, {'name': researcher_2}, 'tutors')
    write_link('Researcher', 'Researcher', {'name': researcher_2}, {'name': researcher_1}, 'tutored by')

100%|██████████| 4/4 [00:00<00:00, 63.45it/s]


# 3. Чтение результатов

In [11]:
nodes_text_properties = {'Speaker': 'code', 'Recording': 'id', 'Researcher': 'name'}
graph_folder = Path('./graphs/')

In [12]:
def visualize_result(query_graph, nodes_text_properties=nodes_text_properties, graph_folder=graph_folder):
    visual_graph = pyvis.network.Network(notebook=True, cdn_resources='remote', directed=True)

    for node in query_graph.nodes:
        node_label = list(node.labels)[0]
        node_text = node[nodes_text_properties[node_label]]
        visual_graph.add_node(node.element_id, str(node_text), group=node_label)

    for relationship in query_graph.relationships:
        visual_graph.add_edge(
            relationship.start_node.element_id,
            relationship.end_node.element_id,
            title=relationship.type
        )
    if not graph_folder.exists():
        graph_folder.mkdir()
    return visual_graph.show(graph_folder.joinpath(f'result_{time.time_ns() // 100 % 10**8}.html').__fspath__(), 
                             notebook=True)

In [13]:
def get_db_graph(driver=driver):
    db_graph = driver.execute_query('''MATCH (n), (n1)-[r]->(n2) RETURN n, n1, n2, r''', result_transformer_=Result.graph)
    return db_graph

In [14]:
visualize_result(get_db_graph())

graphs\result_1469218.html


# 4. Удаление

Удаляю те записи, которые ни с кем не связаны

In [15]:
driver.execute_query('''
MATCH (rec:Recording) 
WHERE NOT (rec)<-[]-() 
DELETE rec
''')
visualize_result(get_db_graph())

graphs\result_23048912.html


# 5. Обновление

В случае, если есть связь руководитель - руководимый, назначаю первому позицию "профессор", а второму -- студент

In [16]:
driver.execute_query('''
MATCH (prof:Researcher)-[:tutors]->(student:Researcher)
SET prof.position = 'professor', student.position = 'student'
''')

EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x0000023E30EB8F50>, keys=[])

In [17]:
result_graph = driver.execute_query('''
MATCH (student:Researcher {position: 'student'})-[rel]-(any)
RETURN student, rel, any 
''', result_transformer_=Result.graph)
visualize_result(result_graph)

graphs\result_23517179.html


# 6. Поиск элемента

Найдем братьев/сестёр среди носителей

In [18]:
result_graph = driver.execute_query('''
MATCH (child_1)-[rel_1:related {type: 'child'}]->(parent), (child_2)-[rel_2:related {type: 'child'}]->(parent)
RETURN child_1, child_2, parent, rel_1, rel_2
''', result_transformer_=Result.graph)
visualize_result(result_graph)

graphs\result_23991410.html


# 7. Сортировка

Выберу записи, сделанные в период с 01.01.2020 до 31.12.2022 и отсортирую по длине

In [19]:
result = driver.execute_query('''
MATCH (rec:Recording)
WHERE date(rec.date) >= date('2020-01-01') AND date(rec.date) <= date('2022-12-31')
RETURN rec
ORDER BY rec.length
''', result_transformer_=Result.data)
pd.DataFrame([res['rec'] for res in result])

,date,length,location,id
0,2020-03-21,157,K,4797
1,2022-08-11,778,H,65919
2,2022-05-07,5909,R,22487
3,2020-01-20,6117,U,38784
4,2022-02-15,7109,D,52571
5,2020-06-07,7269,Q,30654
6,2021-07-14,8393,M,69253
7,2022-05-14,9799,R,53515
8,2022-03-11,11941,H,21468
9,2022-10-31,12995,L,23561


# 8. Агрегация

Посчитаю средний возраст сотрудника ZL

In [20]:
driver.execute_query('''
MATCH (res:Researcher)
WHERE res.organisation = 'ZL'
RETURN avg(res.age)
''', result_transformer_=Result.value)[0]

27.8

# 9. Кратчайший путь

Найду, как и через какого исседователя можно выйти на носителя AIYM, если не обращаться к молодым исследователям (моложе 30) и исследователям из ZL

In [21]:
result_graph = driver.execute_query('''
MATCH p = shortestPath((res:Researcher)-[link*]-(speaker:Speaker {code: 'AIYM'}))
WHERE res.age > 30 and res.organisation <> 'ZL'
RETURN p
''', result_transformer_=Result.graph)
visualize_result(result_graph)

graphs\result_25014296.html


# 10. Закрываю драйвер

In [22]:
driver.close()

In [23]:
# for file in graph_folder.iterdir():
#     file.unlink()